In [ ]:
import json
import os
from pathlib import Path
import torch
import numpy as np
from torch_geometric.data import Data
from rdkit import Chem
from rdkit.Chem import AllChem
import py3Dmol
from IPython.display import display
from tqdm import tqdm

# --- 配置和映射 (与您之前的脚本保持一致) ---
# 这些常量用于将图数据中的编码转换为化学意义
ATOM_MAP = ['H', 'C', 'N', 'O', 'F']
BOND_TYPE_MAP = {
    0: Chem.rdchem.BondType.SINGLE,
    1: Chem.rdchem.BondType.DOUBLE,
    2: Chem.rdchem.BondType.TRIPLE,
    3: Chem.rdchem.BondType.AROMATIC
}
# 第6个原子类型是吸收态/填充节点，在转换时需忽略
ABSORBING_ATOM_TYPE_INDEX = 5
# 第5个键类型是无连接/填充边，在转换时需忽略
NO_BOND_EDGE_TYPE_INDEX = 4


def pyg_to_rdkit_mol(pyg_data):
    """
    将 PyTorch Geometric (PyG) 图数据对象转换为 RDKit 分子对象。
    这个函数与您之前用于QED计算的函数完全相同。
    """
    try:
        mol = Chem.RWMol()
        node_map = {} # 映射 PyG 节点索引到 RDKit 原子索引
        atom_features = pyg_data.x.cpu().numpy()
        atom_types_idx = np.argmax(atom_features, axis=1)

        # 1. 添加原子
        for i, atom_idx in enumerate(atom_types_idx):
            if atom_idx != ABSORBING_ATOM_TYPE_INDEX:
                atom_symbol = ATOM_MAP[atom_idx]
                mol.AddAtom(Chem.Atom(atom_symbol))
                node_map[i] = len(node_map)

        if not node_map: return None

        # 2. 添加化学键
        edge_index = pyg_data.edge_index.cpu().numpy()
        edge_attrs = pyg_data.edge_attr.cpu().numpy()
        bond_types_idx = np.argmax(edge_attrs, axis=1)

        for i in range(edge_index.shape[1]):
            u, v = edge_index[:, i]
            bond_type_idx = bond_types_idx[i]
            
            if u in node_map and v in node_map and bond_type_idx != NO_BOND_EDGE_TYPE_INDEX:
                rdkit_u, rdkit_v = node_map[u], node_map[v]
                if mol.GetBondBetweenAtoms(rdkit_u, rdkit_v) is None:
                    bond_type = BOND_TYPE_MAP.get(bond_type_idx)
                    if bond_type:
                        mol.AddBond(rdkit_u, rdkit_v, bond_type)

        # 3. 清理和验证分子
        final_mol = mol.GetMol()
        Chem.SanitizeMol(final_mol)
        return final_mol
    except Exception:
        return None

def view_directory_of_json_mols(directory_path):
    """
    扫描指定目录下的所有 .json 文件，将其转换为分子并使用 py3Dmol 进行可视化。
    """
    # 1. 检查目录是否存在
    json_dir = Path(directory_path)
    if not json_dir.is_dir():
        print(f"[!] 错误: 找不到目录 '{directory_path}'。")
        return

    # 2. 查找目录中所有的 .json 文件
    json_files = sorted(list(json_dir.glob('*.json')))
    
    if not json_files:
        print(f"[!] 在目录 '{directory_path}' 中没有找到任何 .json 文件。")
        return

    print(f"[✓] 在 '{directory_path}' 中找到 {len(json_files)} 个 .json 文件。正在逐个显示...")
    print("="*50)

    # 3. 遍历、转换并显示每个文件
    for json_path in tqdm(json_files, desc="正在可视化分子"):
        print(f"\n--- 正在处理: {json_path.name} ---")
        
        # 步骤 A: 读取 JSON 文件并创建 PyG Data 对象
        try:
            with open(json_path, 'r') as f:
                mol_dict = json.load(f)
            
            pyg_data = Data(
                x=torch.tensor(mol_dict['x'], dtype=torch.float),
                edge_index=torch.tensor(mol_dict['edge_index'], dtype=torch.long),
                edge_attr=torch.tensor(mol_dict['edge_attr'], dtype=torch.float)
            )
        except Exception as e:
            print(f"  [!] 读取或解析文件 {json_path.name} 时出错: {e}")
            continue

        # 步骤 B: 将 PyG Data 对象转换为 RDKit 分子
        mol = pyg_to_rdkit_mol(pyg_data)
        
        if not mol:
            print("  [!] 无法将此图数据转换为一个有效的 RDKit 分子。")
            continue
            
        # 步骤 C: 为分子生成 3D 构象并进行可视化
        try:
            # 添加氢原子以便生成更准确的3D结构
            mol_with_h = Chem.AddHs(mol)
            # 生成3D坐标
            AllChem.EmbedMolecule(mol_with_h, AllChem.ETKDG())
            # （可选）使用力场优化结构
            AllChem.MMFFOptimizeMolecule(mol_with_h)
            
            # 从 RDKit 分子对象生成 MOL block 字符串
            mol_block = Chem.MolToMolBlock(mol_with_h)

            # 使用 py3Dmol 创建交互式视图
            view = py3Dmol.view(width=500, height=400)
            view.addModel(mol_block, 'mol')
            view.setStyle({'stick': {}, 'sphere': {'scale': 0.25}})
            view.zoomTo()
            
            # 在 Jupyter 环境或类似环境中显示视图
            display(view)

        except Exception as e:
            print(f"  [!] 为 {json_path.name} 生成3D视图时出错: {e}")
    
    print("\n" + "="*50)
    print("[✓] 所有 .json 文件处理完毕。")


if __name__ == '__main__':
    # --- 用户配置区 ---
    # 指定包含 .json 分子数据文件的文件夹路径
    molecules_directory = 'check/data'
    
    # --- 运行查看器 ---
    view_directory_of_json_mols(molecules_directory)

[✓] 在 'check/data' 中找到 19 个 .json 文件。正在逐个显示...


正在可视化分子:   0%|          | 0/19 [00:00<?, ?it/s]


--- 正在处理: graph_100357_subgraph_15.json ---


3Dmol.js failed to load for some reason. Please check your browser console for error messages.


--- 正在处理: graph_100359_subgraph_3.json ---


...

正在可视化分子:  95%|█████████▍| 18/19 [00:00<00:00, 39.19it/s]


--- 正在处理: graph_100528_subgraph_12.json ---


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

正在可视化分子: 100%|██████████| 19/19 [00:00<00:00, 38.29it/s]


[✓] 所有 .json 文件处理完毕。
